This is the second assignment for the Coursera course "Advanced Machine Learning and Signal Processing"

Again, please insert to code to your ApacheCouchDB based Cloudant instance below using the "Insert Code" function of Watson Studio( you've done this in Assignment 1 before)

Done, just execute all cells one after the other and you are done - just note that in the last one you have to update your email address (the one you've used for coursera) and obtain a submission token, you get this from the programming assignment directly on coursera.

Please fill in the sections labelled with "###YOUR_CODE_GOES_HERE###"

In [28]:
#your cloudant credentials go here
# @hidden_cell
credentials_1 = {
  'password':"""f65***""",
  'custom_url':'https://***',
  'username':'bf5***',
  'url':'https://undefined'
}


Let's create a SparkSession object and put the Cloudant credentials into it

In [29]:
spark = SparkSession\
    .builder\
    .appName("Cloudant Spark SQL Example in Python using temp tables")\
    .config("cloudant.host",credentials_1['custom_url'].split('@')[1])\
    .config("cloudant.username", credentials_1['username'])\
    .config("cloudant.password",credentials_1['password'])\
    .config("jsonstore.rdd.partitions", 1)\
    .getOrCreate()

Now it’s time to have a look at the recorded sensor data. You should see data similar to the one exemplified below….


In [30]:
df=spark.read.load('shake_classification', "com.cloudant.spark")

df.createOrReplaceTempView("df")
spark.sql("SELECT * from df").show()


+-----+--------+-----+-----+-----+--------------------+--------------------+
|CLASS|SENSORID|    X|    Y|    Z|                 _id|                _rev|
+-----+--------+-----+-----+-----+--------------------+--------------------+
|    0|asdfghjk|  0.0|  0.0|  0.0|0e68bda4f806a1436...|1-461daac6b553896...|
|    0|asdfghjk| 0.01| 0.01| 0.01|0e68bda4f806a1436...|1-a4b6e8d60269136...|
|    0|asdfghjk| 0.01| 0.01| 0.01|0e68bda4f806a1436...|1-a4b6e8d60269136...|
|    0|asdfghjk|  0.0|  0.0|  0.0|0e68bda4f806a1436...|1-461daac6b553896...|
|    0|asdfghjk|-0.02|-0.02|-0.02|0e68bda4f806a1436...|1-fbc3c86a654a820...|
|    0|asdfghjk| 0.01| 0.01| 0.01|0e68bda4f806a1436...|1-a4b6e8d60269136...|
|    0|asdfghjk| 0.01| 0.01| 0.01|0e68bda4f806a1436...|1-a4b6e8d60269136...|
|    0|asdfghjk|  0.0|  0.0|  0.0|0e68bda4f806a1436...|1-461daac6b553896...|
|    0|asdfghjk|-0.02|-0.02|-0.02|0e68bda4f806a1436...|1-fbc3c86a654a820...|
|    0|asdfghjk| 0.01| 0.01| 0.01|0e68bda4f806a1436...|1-a4b6e8d60269136...|

Please create a VectorAssembler which consumed columns X, Y and Z and produces a column “features”


In [31]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Normalizer

indexer = StringIndexer(inputCol="CLASS", outputCol="label")
encoder = OneHotEncoder(inputCol="label", outputCol="labelVec")
vectorAssembler = VectorAssembler(inputCols=["X","Y","Z"],
                                  outputCol="features")
normalizer = Normalizer(inputCol="features", outputCol="features_norm", p=1.0)

Please insatiate a classifier from the SparkML package and assign it to the classifier variable. Make sure to either
1.	Rename the “CLASS” column to “label” or
2.	Specify the label-column correctly to be “CLASS”


In [55]:
# LogisticReg accuracy was 54.7% Naive Bayes requires non-negative data
# https://spark.apache.org/docs/2.3.0/ml-classification-regression.html#gradient-boosted-tree-classifier
# from pyspark.ml.classification import LogisticRegression
# classifier = LogisticRegression(maxIter=200, regParam=0.2, elasticNetParam=0.8)

from pyspark.ml.classification import GBTClassifier
classifier = GBTClassifier(labelCol="label", featuresCol="features_norm", maxIter=10)

Let’s train and evaluate…


In [56]:
from pyspark.ml import Pipeline
# pipeline = Pipeline(stages=[vectorAssembler, classifier])
pipeline = Pipeline(stages=[indexer, encoder, vectorAssembler, normalizer,classifier])

In [57]:
model = pipeline.fit(df)

In [58]:
prediction = model.transform(df)

In [59]:
prediction.show()

+-----+--------+-----+-----+-----+--------------------+--------------------+-----+-------------+-------------------+--------------------+----------+
|CLASS|SENSORID|    X|    Y|    Z|                 _id|                _rev|label|     labelVec|           features|       features_norm|prediction|
+-----+--------+-----+-----+-----+--------------------+--------------------+-----+-------------+-------------------+--------------------+----------+
|    0|asdfghjk|  0.0|  0.0|  0.0|0e68bda4f806a1436...|1-461daac6b553896...|  0.0|(1,[0],[1.0])|          (3,[],[])|           (3,[],[])|       0.0|
|    0|asdfghjk| 0.01| 0.01| 0.01|0e68bda4f806a1436...|1-a4b6e8d60269136...|  0.0|(1,[0],[1.0])|   [0.01,0.01,0.01]|[0.33333333333333...|       0.0|
|    0|asdfghjk| 0.01| 0.01| 0.01|0e68bda4f806a1436...|1-a4b6e8d60269136...|  0.0|(1,[0],[1.0])|   [0.01,0.01,0.01]|[0.33333333333333...|       0.0|
|    0|asdfghjk|  0.0|  0.0|  0.0|0e68bda4f806a1436...|1-461daac6b553896...|  0.0|(1,[0],[1.0])|          

In [60]:
prediction.printSchema()

root
 |-- CLASS: long (nullable = true)
 |-- SENSORID: string (nullable = true)
 |-- X: double (nullable = true)
 |-- Y: double (nullable = true)
 |-- Z: double (nullable = true)
 |-- _id: string (nullable = true)
 |-- _rev: string (nullable = true)
 |-- label: double (nullable = true)
 |-- labelVec: vector (nullable = true)
 |-- features: vector (nullable = true)
 |-- features_norm: vector (nullable = true)
 |-- prediction: double (nullable = true)



In [61]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
binEval = MulticlassClassificationEvaluator().setMetricName("accuracy") .setPredictionCol("prediction").setLabelCol("label")
    
binEval.evaluate(prediction) 

0.822438717787555

If you are happy with the result (I’m happy with > 0.55) please submit your solution to the grader by executing the following cells, please don’t forget to obtain an assignment submission token (secret) from the Courera’s graders web page and paste it to the “secret” variable below, including your email address you’ve used for Coursera. (0.55 means that you are performing better than random guesses)


In [62]:
!rm -Rf a2_m2.json

In [63]:
prediction = prediction.repartition(1)
prediction.write.json('a2_m2.json')

In [64]:
!rm -f rklib.py
!wget https://raw.githubusercontent.com/romeokienzler/developerWorks/master/coursera/ai/rklib.py

--2018-12-28 15:48:04--  https://raw.githubusercontent.com/romeokienzler/developerWorks/master/coursera/ai/rklib.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2289 (2.2K) [text/plain]
Saving to: ‘rklib.py’

100%[======================================>] 2,289       --.-K/s   in 0s      

2018-12-28 15:48:05 (11.1 MB/s) - ‘rklib.py’ saved [2289/2289]



In [65]:
!zip -r a2_m2.json.zip a2_m2.json

updating: a2_m2.json/ (stored 0%)
updating: a2_m2.json/_SUCCESS (stored 0%)
updating: a2_m2.json/._SUCCESS.crc (stored 0%)
  adding: a2_m2.json/.part-00000-77d08854-3b0d-49e6-aa83-0989b598b38b.json.crc (stored 0%)
  adding: a2_m2.json/part-00000-77d08854-3b0d-49e6-aa83-0989b598b38b.json (deflated 93%)


In [66]:
!base64 a2_m2.json.zip > a2_m2.json.zip.base64

In [67]:
from rklib import submit
key = "J3sDL2J8EeiaXhILFWw2-g"
part = "G4P6f"
email = ""
secret = ""

with open('a2_m2.json.zip.base64', 'r') as myfile:
    data=myfile.read()
submit(email, secret, key, part, [part], data)

Submission successful, please check on the coursera grader page for the status
-------------------------
{"elements":[{"itemId":"LTL4F","id":"f_F-qCtuEei_fRLwaVDk3g~LTL4F~Y22ObgrqEemNsgqSGaU9BA","courseId":"f_F-qCtuEei_fRLwaVDk3g"}],"paging":{},"linked":{}}
-------------------------
